В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [2]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [4]:
dataname = 'adult_ON_QnSC_200k'
dataname_initial = 'adult'
model_short = 'tabddpm_ON_QnSC'
model_name = 'tabddpm_ON_QnSC'
model_name_postfix = '_200k'

idx = list(range(10))
constants = [0, 0.001, 0.01, 0.1, 0.25, 0.5]

CONFIG.add_arg('save_cat', None)
target_cat=True

In [5]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [6]:
model_save_path_hist = []
for sigma in constants:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i in idx:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [7]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/adult_ON_QnSC_200k` folder


In [10]:
CONFIG.get_all_args()

{'dataname': 'adult_ON_QnSC_200k',
 'method': None,
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': None,
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 103,
 'real_path': './synthetic/adult_ON_QnSC_200k/initial_real.csv',
 'test_path': './synthetic/adult_ON_QnSC_200k/initial_test.csv',
 'info_path': './data/adult_ON_QnSC_200k/initial_info.json',
 'save_path': None,
 'num_clusters': 25,
 'save_cat': None}

### Sample + Eval

In [ ]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers, target_cat=target_cat)

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=True, value=' >50K') # ошибка
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

./models/tabddpm_ON_QnSC/ckpt/adult/tabddpm_ON_QnSC_200k_const_0 tabddpm_ON_QnSC_200k_const_0 ./synthetic/adult_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Shape torch.Size([32561, 109])
Sampling time: 191.06463503837585
Sample path changed to ./synthetic/adult_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_const_0.csv


/venv/main/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 71.69it/s]|
Column Shapes Score: 98.32%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:01<00:00, 66.76it/s]|
Column Pair Trends Score: 96.79%

Overall Score (Average): 97.56%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_200K_CONST_0: 1.675 ± 0.983
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_200K_CONST_0: 3.207 ± 1.547


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


...

Saving scores to  eval/mle/adult_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_200K_CONST_0: 0.871 ± 0.007
adult_ON_QnSC_200k, tabddpm_ON_QnSC_200k_const_0: 0.9281485211032071
n_clusters: 25
n_clusters: 25
n_clusters: 25
DCR Score, a value closer to 0.5 is better
adult_ON_QnSC_200k-tabddpm_ON_QnSC_200k_const_0, DCR Score = 0.5031479377169006
=========== All Features ===========
Data shape:  (32561, 110)
alpha precision: 0.940598, beta recall: 0.478433
./models/tabddpm_ON_QnSC/ckpt/adult/tabddpm_ON_QnSC_200k_const_0_001 tabddpm_ON_QnSC_200k_const_0_001 ./synthetic/adult_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0_001.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Shape torch.Size([32561, 109])
Sampling time: 191.29862451553345
Sample path changed to ./synthetic/adult_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_const_0_001.csv
TABDDPM_ON_QNSC_200K_C

DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:11<00:00,  3.27it/s]


Saving scores to  eval/mle/adult_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0_001.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_200K_CONST_0_001: 0.877 ± 0.006


In [13]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname_initial}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [16]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4).drop(columns=('base_metrics', 0)).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

\
             Model                              Type   Data   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  adult   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  adult   
0  tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  adult   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  adult   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  adult   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  adult   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  adult   
0  tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  adult   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  adult   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  adult   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  adult   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  adult   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  adult   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  adult   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  adult   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  adult   

              similarity                              \
  Column Shapes Score, % Column Pair Trends Score, %   
0                98.7070                     97.3548   
0                98.6542                     97.2714   
0                98.5662                     97.2887   
0                98.5289                     97.2021   
0                98.4693                     97.0452   
0                98.4392                     96.8986   
0                98.4822                     96.8074   
0                98.3248                     96.7932   
0                98.3244                     96.7914   
0                98.1751                     96.6069   
0                98.0766                     96.5317   
0                98.1831                     96.2898   
0                98.1401                     96.2726   
0                98.1080                     95.8018   
0                97.8486                     95.9311   
0                97.4282                     94.7817   

                              \
  Overall Score (Average), %   
0                    98.0309   
0                    97.9628   
0                    97.9274   
0                    97.8655   
0                    97.7573   
0                    97.6689   
0                    97.6448   
0                    97.5590   
0                    97.5579   
0                    97.3910   
0                    97.3042   
0                    97.2364   
0                    97.2063   
0                    96.9549   
0                    96.8898   
0                    96.1050   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              1.2930   
0                                              1.3458   
0                                              1.4338   
0                                              1.4711   
0                                              1.5307   
0                                              1.5608   
0                                              1.5178   
0                                              1.6752   
0                                              1.6756   
0                                              1.8249   
0                                              1.9234   
0                                              1.8169   
0                                              1.8599   
0                                              1.8920   
0                                              2.1514   
0                                              2.5718   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                  0.7684   
0                                                  0.8565   
0                                                  0.7373   
0       

In [17]:
import os
if not os.path.exists(f'./eval/total/{dataname_initial}'):
    os.makedirs(f'./eval/total/{dataname_initial}')
final_metrics_table.to_csv(f'./eval/total/{dataname_initial}/{model_short}{model_name_postfix}_final_metrics_table.csv', index=False)